# Colour Correction using Splines

### Define constants

In [5]:
%load_ext autoreload
%autoreload 2
import colour
from data import load_dataset_sfu, load_dataset_csv, load_cmfs, load_camera, load_insitu, msds_to_rgb, msds_to_xyz, load_dataset_skin
import numpy as np

RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
    
SFU_FILE_PATH = 'data/reflect_db.reflect'
CAVE_FOSTER2004_PATH = 'data/cave_foster2002.csv'
INSITU_PATH = "data/insitu_dataset.csv"
CAVE_PATH = 'data/cave.csv'
FOSTER_50_PATH = 'data/foster50.csv'
CAMERA = 'sigma'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

# sfu_dataset = load_dataset_sfu(SFU_FILE_PATH)
cave_foster2004_dataset = load_dataset_csv(CAVE_FOSTER2004_PATH)
foster_50_dataset = load_dataset_csv(FOSTER_50_PATH)
insitu = load_insitu(INSITU_PATH)
# cave = load_dataset_csv(CAVE_PATH)
# skin = load_dataset_skin()


In [7]:
from plotting import plot_chromaticity_diagram
TRAIN = cave_foster2004_dataset
TEST = foster_50_dataset
VALIDATION = insitu

### Computing Observer Responses
We can easily change the order of test and train sets here

In [8]:
from plotting import plot_chromaticity_diagram

cmfs, illuminant = load_cmfs()
response_trainset_xyz = msds_to_xyz(TRAIN, cmfs, illuminant)
response_testset_xyz = msds_to_xyz(TEST, cmfs, illuminant)
response_validation_xyz = msds_to_xyz(VALIDATION, cmfs, illuminant)

[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]


### Computing Camera Responses

In [10]:
MSDS_TRAIN = load_camera(CAMERA)
response_trainset_camera = msds_to_rgb(TRAIN,MSDS_TRAIN, illuminant)
response_testset_camera = msds_to_rgb(TEST,MSDS_TRAIN, illuminant)
response_validationset_camera = msds_to_rgb(VALIDATION,MSDS_TRAIN, illuminant)

[  938.7258382   1714.98576086  1939.82119587]
[  938.7258382   1714.98576086  1939.82119587]
[  938.7258382   1714.98576086  1939.82119587]


### Fit Generalized Additive Model with P-splines

### Nikon

In [13]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
np.int = np.int64
if CAMERA == 'nikon':
    # gam = GAMOptimizer(lams=1e-6, order=3, n_splines=5)
    
    # gam = GAMOptimizer(lams=0.0001,order=3, n_splines=10)
    gam = GAMOptimizer(lams=0.01,order=3, n_splines=20)

    gam.fit(response_trainset_camera, response_trainset_xyz)

    pred(gam, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

[autoreload of scipy.special._basic failed: Traceback (most recent call last):
  File "/Users/jsuominen/anaconda3/envs/mps/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/jsuominen/anaconda3/envs/mps/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/jsuominen/anaconda3/envs/mps/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/jsuominen/anaconda3/envs/mps/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 309, in update_function
    setattr(old, name, getattr(new, name))
ValueError: comb() requires a code object with 5 free vars, not 0
]
[autoreload of scipy.linalg._decomp failed: Traceback (most recent call last):
  File "/Users/jsuominen/anaconda3/envs/mps/lib/python3.10/site-packages/IPython/extensions/autorel

ModuleNotFoundError: No module named 'matplotlib.gridspec'; 'matplotlib' is not a package

### Sigma

In [ ]:
from sklearn.metrics import make_scorer
from colour_math import deltae_mean
from models import GAMOptimizer
from evaluate import pred
np.int = np.int64
if CAMERA == 'sigma':
    # gam = GAMOptimizer(lams=1e-9, order=3, n_splines=5)
    # gam = GAMOptimizer(lams=0.0001,order=3, n_splines=10)
    gam = GAMOptimizer(lams=0.0001,order=3, n_splines=20)

    gam.fit(response_trainset_camera, response_trainset_xyz)

    pred(gam, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

3600
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.73
DeltaE max: 16.95
DeltaE median: 1.46
DeltaE 95 percentile: 4.00
DeltaE 99 percentile: 5.94


### Fit Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression(fit_intercept=False)


linear.fit(response_trainset_camera, response_trainset_xyz)
pred(linear, response_testset_camera, response_testset_xyz, "Foster 50")


---- RESULTS Foster 50 ----
DeltaE mean: 2.20
DeltaE max: 23.49
DeltaE median: 1.52
DeltaE 95 percentile: 6.98
DeltaE 99 percentile: 10.56


In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=1))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 2.6196741857
        x: [ 8.371e-01 -5.934e-01  6.989e-01 -4.839e-01  1.788e+00
            -2.987e-01  9.454e-01 -4.451e+00  4.568e+00]
      nit: 30
      jac: [-6.855e-07 -2.384e-07  1.192e-07  2.116e-06  1.281e-06
             8.941e-07 -1.103e-06 -7.153e-07 -5.662e-07]
 hess_inv: [[ 1.267e+00 -2.835e+00 ..., -2.499e+00  1.543e+00]
            [-2.835e+00  7.029e+00 ...,  6.286e+00 -4.030e+00]
            ..., 
            [-2.499e+00  6.286e+00 ...,  1.043e+01 -6.902e+00]
            [ 1.543e+00 -4.030e+00 ..., -6.902e+00  4.731e+00]]
     nfev: 370
     njev: 37
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.57
DeltaE max: 19.60
DeltaE median: 2.15
DeltaE 95 percentile: 5.59
DeltaE 99 percentile: 7.84


### Fit 3rd order Root-Polynomial Model

In [ ]:
from models import PolynomialTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])

RP_linear_3.fit(response_trainset_camera, response_trainset_xyz)

pred(RP_linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.32
DeltaE max: 34.32
DeltaE median: 1.65
DeltaE 95 percentile: 6.90
DeltaE 99 percentile: 11.37


### Fit a 2nd order Root-Polynomial Model

In [ ]:
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_2 = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])


RP_linear_2.fit(response_trainset_camera, response_trainset_xyz)
pred(RP_linear_2, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.17
DeltaE max: 25.38
DeltaE median: 1.50
DeltaE 95 percentile: 6.84
DeltaE 99 percentile: 10.04


In [ ]:
P_Linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_Linear_3.fit(response_trainset_camera, response_trainset_xyz)
pred(P_Linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.92
DeltaE max: 20.78
DeltaE median: 1.49
DeltaE 95 percentile: 5.01
DeltaE 99 percentile: 8.16


In [ ]:
P_linear = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_linear.fit(response_trainset_camera, response_trainset_xyz)
pred(P_linear, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.02
DeltaE max: 22.07
DeltaE median: 1.49
DeltaE 95 percentile: 5.82
DeltaE 99 percentile: 9.20


In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000P = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=3))
])


DE2000P.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000P, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 1.53974279145
        x: [ 8.419e-01 -6.158e-01 ..., -7.217e+00  2.435e+00]
      nit: 247
      jac: [-1.507e-03 -1.429e-03 ..., -3.873e-04 -3.987e-04]
 hess_inv: [[ 2.252e+00 -4.106e+00 ..., -1.094e+01 -1.686e+00]
            [-4.106e+00  9.558e+00 ...,  1.947e+00  1.647e+01]
            ..., 
            [-1.094e+01  1.947e+00 ...,  3.280e+03 -1.957e+03]
            [-1.686e+00  1.647e+01 ..., -1.957e+03  1.573e+03]]
     nfev: 17869
     njev: 308
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.98
DeltaE max: 17.92
DeltaE median: 1.74
DeltaE 95 percentile: 4.42
DeltaE 99 percentile: 6.38


In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=True, degree=3))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 1.608765511
        x: [ 7.303e+01 -4.322e+02 ..., -6.789e+01  1.045e+02]
      nit: 343
      jac: [-5.868e-04 -5.814e-04 ...,  1.741e-04  2.467e-04]
 hess_inv: [[ 2.178e+03  2.090e+03 ...,  1.096e+03 -4.136e+03]
            [ 2.090e+03  3.924e+03 ...,  1.335e+03 -6.900e+03]
            ..., 
            [ 1.095e+03  1.335e+03 ...,  8.042e+03 -3.848e+03]
            [-4.137e+03 -6.902e+03 ..., -3.850e+03  1.726e+04]]
     nfev: 15880
     njev: 397
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 2.02
DeltaE max: 43.46
DeltaE median: 1.36
DeltaE 95 percentile: 5.54
DeltaE 99 percentile: 9.71
